## Query OpenAlex for works and theire Open Access Status authored by a person
This notebook queries the [OpenAlex API](https://docs.openalex.org/api) via its `/works` endpoint for works authored by a person. It takes an ORCID URL from a list of ORCID IDs as input which is used to filter for works where '`authorships.author.orcid`' matches the given ORCID URL.
The notebook iterates through the given list and displys all DOIs and theire Open Access Status.

In [1]:
# Prerequisites:
import requests         # dependency to make HTTP calls

In [2]:
list_of_ids=["0000-0001-5406-9458", "0000-0001-6604-6253","0000-0001-8343-8654"]

We use it to query the OpenAlex API for works that specified the ORCID URL within their metadata in the field '`authorships.author.orcid`'.
 Since the API uses [pagination](https://docs.openalex.org/api/get-lists-of-entities#pagination), we need to loop through all pages to get the complete result set.

In [3]:
# OpenAlex endpoint to query for works
OPENALEX_API_WORKS = "https://api.openalex.org/works"

# query all works that are connected to orcid
def query_openalex_for_person2works(orcid):
    page = 1
    max_page = 1
    
    while page <= max_page:
        params = {'filter': 'authorships.author.orcid:'+orcid, 'page': page}
        response = requests.get(url=OPENALEX_API_WORKS,
                                params=params,
                                headers= {'Accept': 'application/json'})
        response.raise_for_status()
        result=response.json()

        # calculate max page number in first loop
        if max_page == 1:
            max_page = determine_max_page(result)
        page = page + 1
        yield result

# calculate max number of result pages
def determine_max_page(response_data):
    item_count = response_data['meta']['count']
    items_per_page = response_data['meta']['per_page']
    max_page_ceil = item_count // items_per_page + bool(item_count % items_per_page)
    return max_page_ceil

From the resulting list of works we extract and print out title and DOI. 

*Note: works that do not have a DOI assigned, will not be printed.*

In [4]:
# from the result pages we get from the OpenAlex API, extract the data about works
def extract_works_from_page(page):
    return [work for work in page.get('results') or []]

# extract DOI from work
def extract_doi(work):
    doi=work.get('ids', {}).get('doi') or ""
    doi_id=doi.replace("https://doi.org/", "") if doi else doi
    title=work.get('display_name', "")
    oa=work.get('open_access', {}).get('oa_status')
    return doi_id, title, oa

def main_search(orcid):
    list_of_pages=query_openalex_for_person2works(orcid)
    for page in list_of_pages or []:
        works=extract_works_from_page(page)
        for work in works or []:
            doi,title,oa=extract_doi(work)
            if doi:
                print(f"{doi}, #, {title}, # Open Access Status:, {oa}")
                
# main programm:
for item in list_of_ids:
    main_search(item)                

10.1111/jdv.13854, #, A systematic review of worldwide epidemiology of psoriasis, # Open Access Status:, closed
10.1186/1475-2891-12-103, #, Milk is not just food but most likely a genetic transfection system activating mTORC1 signaling for postnatal growth, # Open Access Status:, gold
10.1016/j.jaci.2004.05.054, #, Decreasing incidence of occupational contact urticaria caused by natural rubber latex allergy in German health care workers, # Open Access Status:, closed
10.1186/1479-5876-12-43, #, Milk: an exosomal microRNA transmitter promoting thymic regulatory T cell maturation preventing the development of atopy?, # Open Access Status:, gold
10.1111/bjd.12083, #, Impact of atopic dermatitis and loss-of-function mutations in the filaggrin gene on the development of occupational irritant contact dermatitis, # Open Access Status:, green
10.1111/jdv.14423, #, European Surveillance System on Contact Allergies (ESSCA): results with the European baseline series, 2013/14, # Open Access Statu

10.1111/jdv.13602, #, Non-melanoma skin cancer by solar UV: the neglected occupational threat, # Open Access Status:, bronze
10.3390/biom11060851, #, Exosome-Derived MicroRNAs of Human Milk and Their Effects on Infant Health and Development, # Open Access Status:, gold
10.1007/s00105-008-1707-8, #, Neue Formen der Zusammenarbeit zwischen Praxis und Klinik, # Open Access Status:, closed
10.1111/jdv.16303, #, Personal solar ultraviolet radiation dosimetry in an occupational setting across Europe, # Open Access Status:, green
10.1111/bjd.19152, #, Global evidence on occupational sun exposure and keratinocyte cancers: a systematic review, # Open Access Status:, hybrid
10.1111/cod.12140, #, Development of a standardized testing procedure for assessing the irritation potential of occupational skin cleansers, # Open Access Status:, closed
10.1111/jdv.15768, #, Skin cancer in outdoor workers exposed to solar radiation: a largely underreported occupational disease in Italy, # Open Access Status

10.1111/jdv.14276, #, Occupational skin diseases - Development and Implementation of European Standards on Prevention of Occupational Skin Diseases, # Open Access Status:, closed
10.1111/bjd.17579, #, Isotretinoin's paradoxical effects in immortalized sebocytes, # Open Access Status:, closed
10.1111/jdv.16130, #, Assessment of occupational exposure and spectrum of contact sensitization in metalworkers with occupational dermatitis: results of a cohort study within the OCCUDERM project, # Open Access Status:, hybrid
10.1159/000099994, #, Skin Protection for Hairdressers, # Open Access Status:, closed
10.1111/ddg.12348, #, Cooperation between the occupational health insurance and physicians practicing occupational dermatology: Optimization potential in quality assurance, # Open Access Status:, closed
10.1034/j.1600-0536.2002.460413.x, #, Ammonium thiolactate and thiolactic acid: important hairdressers' allergens?, # Open Access Status:, closed
10.1111/srt.12113, #, Standardization of skin

10.1007/s15006-021-9652-x, #, Gesundheitsrisiken durch Milchkonsum, # Open Access Status:, closed
10.1186/s13643-021-01687-5, #, Illness perceptions of adults with eczematous skin diseases: a systematic mixed studies review, # Open Access Status:, gold
10.1111/cod.13912, #, Partially airborne allergic contact dermatitis to rubber additives in a car mechanic, # Open Access Status:, hybrid
10.1186/s12995-021-00314-2, #, New methods for assessing secondary performance attributes of sunscreens suitable for professional outdoor work, # Open Access Status:, gold
10.1111/jdv.17483, #, Patch test informed consent form: position statement by European Academy of Dermatology and Venereology Task Force on Contact Dermatitis, # Open Access Status:, green
10.1111/phpp.12717, #, Stratum corneum biomarkers after in vivo repeated exposure to sub‐erythemal dosages of ultraviolet radiation in unprotected and sunscreen (SPF 50+) protected skin, # Open Access Status:, closed
10.3389/fpubh.2021.756566, #, P

10.1111/jam.15399, #, Novel insights into pivotal risk factors for rectal carriage of extended‐spectrum‐β‐lactamase‐producing enterobacterales within the general population in Lower Saxony, Germany, # Open Access Status:, hybrid
10.1155/2021/5558694, #, UV-Induced Skin Cancer Knowledge, Sun Exposure, and Tanning Behavior among University Students: Investigation of an Opportunity Sample of German University Students, # Open Access Status:, gold
10.2196/preprints.31980, #, The Expressiveness of SNOMED CT for Wound Care: Mapping a standardised Item Set for Leg Ulcers to SNOMED CT (Preprint), # Open Access Status:, closed
10.1016/j.orthtr.2022.03.012, #, Influence of physical activity on well-being at times of the COVID-19 pandemic: a review, # Open Access Status:, bronze
10.1007/s11553-022-00940-6, #, Prävention von berufsbedingtem Hautkrebs bei Beschäftigten im Freien: Entwicklung eines Curriculums zur Fortbildung von Multiplikator*innen, # Open Access Status:, hybrid
10.1136/bmjopen-202

10.4161/hv.29241, #, Vaccines against human diarrheal pathogens, # Open Access Status:, bronze
10.1371/journal.ppat.1006559, #, The plasmid-encoded Ipf and Klf fimbriae display different expression and varying roles in the virulence of Salmonella enterica serovar Infantis in mouse vs. avian hosts, # Open Access Status:, gold
10.1111/cmi.13155, #, Presence of SopE and mode of infection result in increased <i>Salmonella</i> ‐containing vacuole damage and cytosolic release during host cell infection by <scp> <i>Salmonella enterica</i> </scp>, # Open Access Status:, gold
10.1371/journal.pone.0035004, #, Functional Dissection of SseF, a Membrane-Integral Effector Protein of Intracellular Salmonella enterica, # Open Access Status:, gold
10.1515/hsz-2012-0324, #, Imaging the invisible: resolving cellular microcompartments by superresolution microscopy techniques, # Open Access Status:, closed
10.1002/cyto.a.21122, #, Comparison of parameter-adapted segmentation methods for fluorescence microg

10.1038/s41586-018-0507-2, #, Publisher Correction: Neonatal selection by Toll-like receptor 5 influences long-term gut microbiota composition, # Open Access Status:, bronze
10.1101/832675, #, Blocks in tricarboxylic acid cycle ofSalmonella entericacause global perturbation of carbon storage, motility and host-pathogen-interaction, # Open Access Status:, green
10.1128/msphere.00796-19, #, Blocks in Tricarboxylic Acid Cycle of Salmonella enterica Cause Global Perturbation of Carbon Storage, Motility, and Host-Pathogen Interaction, # Open Access Status:, gold
10.1101/2020.01.25.919720, #, Recombinant protein production and purification of SiiD, SiiE and SiiF - components of the SPI4-encoded type I secretion system from Salmonella Typhimurium, # Open Access Status:, green
10.1101/2020.04.03.024968, #, Factors required for adhesion of Salmonella enterica serovar Typhimurium to lettuce (Lactuca sativa), # Open Access Status:, green
10.1101/2020.09.14.297432, #, Exposure to stressors and ant

10.17877/de290r-13911, #, Unterrichtliche Vorgehensweisen bei der Behandlung des kleinen Einmaleins, # Open Access Status:, None
10.17877/de290r-1066, #, Mathematische Lerngelegenheiten bei Würfelspielen – Eine Videoanalyse im Rahmen der Interventionsstudie MaBiiS, # Open Access Status:, None
10.17877/de290r-916, #, Strategieverwendung bei Aufgaben zum kleinen Einmaleins, # Open Access Status:, None
10.17877/de290r-7827, #, Mathematische Kompetenzen wahrnehmen und fördern - von Anfang an, # Open Access Status:, None
10.17877/de290r-17516, #, Begriffsverständnis von Viereck und Dreieck bei Kindern im Alter von 4 bis 6 Jahren – eine Pilotstudie, # Open Access Status:, None
10.1007/978-3-319-78220-1_15, #, Correction to: Mathematics Education Competence of Professionals in Early Childhood Education: A Theory-Based Competence Model, # Open Access Status:, bronze
10.1007/s13138-018-00139-3, #, „Ergänzen mit Erweitern“ und „Abziehen mit Entbündeln“ – Eine explorative Studie zu spezifischen F